In [97]:
import pandas as pd

In [98]:
df = pd.read_csv('precipitation_data.csv', sep='\t')
df.rename(columns={'DateTime': 'datetime', "Unnamed: 2": "precipitation"}, inplace=True)
df.drop(columns=['Unnamed: 1'], inplace=True)
df['datetime'] = df['datetime'].str[:-3]
df['datetime'] = pd.to_datetime(df['datetime'], format='%d/%m/%y %H:%M:%S')
df.head()

,datetime,precipitation
0,2023-01-07 17:07:13,0.2
1,2023-01-07 17:12:34,0.2
2,2023-01-07 17:23:01,0.2
3,2023-01-07 18:56:21,0.2
4,2023-01-07 19:02:41,0.2


In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1842 entries, 0 to 1841
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   datetime       1842 non-null   datetime64[ns]
 1   precipitation  1842 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 28.9 KB


In [100]:
from datetime import timedelta

def compute(row):
    minute_zero = row.replace(second=0).replace(minute=0)
    # row['minute'] = row['datetime'].minute
    if row.minute <= 30:
        new_datetime = minute_zero + timedelta(minutes=30)
    else:
        new_datetime = minute_zero + timedelta(hours=1)
    return new_datetime

df['new_datetime'] = df['datetime'].apply(lambda x: compute(x))
df

,datetime,precipitation,new_datetime
0,2023-01-07 17:07:13,0.2,2023-01-07 17:30:00
1,2023-01-07 17:12:34,0.2,2023-01-07 17:30:00
2,2023-01-07 17:23:01,0.2,2023-01-07 17:30:00
3,2023-01-07 18:56:21,0.2,2023-01-07 19:00:00
4,2023-01-07 19:02:41,0.2,2023-01-07 19:30:00
...,...,...,...
1837,2023-08-14 23:15:59,0.2,2023-08-14 23:30:00
1838,2023-08-14 23:19:28,0.2,2023-08-14 23:30:00
1839,2023-08-14 23:21:33,0.2,2023-08-14 23:30:00
1840,2023-08-14 23:23:42,0.2,2023-08-14 23:30:00


In [101]:
df['new_datetime']

0      2023-01-07 17:30:00
1      2023-01-07 17:30:00
2      2023-01-07 17:30:00
3      2023-01-07 19:00:00
4      2023-01-07 19:30:00
               ...        
1837   2023-08-14 23:30:00
1838   2023-08-14 23:30:00
1839   2023-08-14 23:30:00
1840   2023-08-14 23:30:00
1841   2023-08-16 07:30:00
Name: new_datetime, Length: 1842, dtype: datetime64[ns]

In [105]:
df = df.groupby(df['new_datetime'])['precipitation'].sum().reset_index()

## Filling the missing gaps

In [107]:
# Create a complete range of 30-minute intervals
start_date = df['new_datetime'].min()
end_date = df['new_datetime'].max()

full_range = pd.date_range(start=start_date, end=end_date, freq='30T')
full_range

/var/folders/dq/43_5rtyn5lz1p4vkq89ykdw40000gn/T/ipykernel_70047/3201747488.py:5: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  full_range = pd.date_range(start=start_date, end=end_date, freq='30T')


DatetimeIndex(['2023-01-07 17:30:00', '2023-01-07 18:00:00',
               '2023-01-07 18:30:00', '2023-01-07 19:00:00',
               '2023-01-07 19:30:00', '2023-01-07 20:00:00',
               '2023-01-07 20:30:00', '2023-01-07 21:00:00',
               '2023-01-07 21:30:00', '2023-01-07 22:00:00',
               ...
               '2023-08-16 03:00:00', '2023-08-16 03:30:00',
               '2023-08-16 04:00:00', '2023-08-16 04:30:00',
               '2023-08-16 05:00:00', '2023-08-16 05:30:00',
               '2023-08-16 06:00:00', '2023-08-16 06:30:00',
               '2023-08-16 07:00:00', '2023-08-16 07:30:00'],
              dtype='datetime64[ns]', length=10589, freq='30min')

In [108]:
# Reindex the DataFrame to include all intervals
df.set_index('new_datetime', inplace=True)
df = df.reindex(full_range, fill_value=0).reset_index()
df.rename(columns={'index': 'new_datetime'}, inplace=True)
df

,new_datetime,precipitation
0,2023-01-07 17:30:00,0.6
1,2023-01-07 18:00:00,0.0
2,2023-01-07 18:30:00,0.0
3,2023-01-07 19:00:00,0.2
4,2023-01-07 19:30:00,0.2
...,...,...
10584,2023-08-16 05:30:00,0.0
10585,2023-08-16 06:00:00,0.0
10586,2023-08-16 06:30:00,0.0
10587,2023-08-16 07:00:00,0.0


In [109]:
df.to_excel('result.xlsx')